In [7]:
import TSeriesPreproccesing as TSPP
import SequencePreprocessing as SP
import ClusterProcessing as CP
from importlib import reload
import matplotlib.pyplot as plt
import numpy as np
import utils as utils
import seaborn as sns
from tslearn.clustering import TimeSeriesKMeans
import os 
import pandas as pd
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [8]:
reload(TSPP)
reload(SP)
reload(CP)
pd.set_option('display.max_columns', 100)

In [ ]:
# tickers = ['aapl','nvda','amd','msft']
tickers: ['spy', 'qqq', 'nvda', 'aapl']
start = '2015-01-01'
target_cols = ['sumPctChgclose_1','sumPctChgclose_2','sumPctChgclose_3','sumPctChgclose_4','sumPctChgclose_5','sumPctChgclose_6']
n_steps = 20
interval = '1d'
cluster_features = ['sumPctChgVol_3','sumPctChgClCl_3']
group_params = CP.StockClusterGroupParams(start_date = start, tickers = tickers, interval = interval, target_cols = target_cols, n_steps = n_steps,cluster_features = cluster_features)

In [21]:
cluster_group = CP.StockClusterGroup(group_params)
cluster_group.create_data_set()
cluster_group.create_sequence_set()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
global
-13.435365183856728 14.420677305858975
allgood
rolling


In [22]:
X_train, y_train, X_test, y_test = cluster_group.get_3d_array()
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4640, 10, 81)
(4640, 6)
(1116, 10, 81)
(1116, 6)


In [23]:
# df = cluster_group.data_set.training_dfs[0][target_cols]
# for column in df.columns:
#     sns.histplot(df[column], kde=True, bins=10, label=column)
# print(df.describe())

In [24]:
cluster_group.run_clustering()

(4640, 10, 2)
(1116, 10, 2)


In [25]:
reload(CP)
cluster_group.create_clusters()

In [26]:
from collections import Counter
Counter(cluster_group.test_labels)

Counter({15: 104,
         5: 72,
         9: 74,
         4: 25,
         2: 77,
         3: 120,
         12: 62,
         14: 143,
         13: 116,
         11: 95,
         1: 68,
         7: 40,
         16: 45,
         8: 33,
         10: 10,
         6: 25,
         0: 7})

In [28]:
cluster_group.display_all_clusters()

In [29]:
cluster_group.clusters[1].visualize_cluster()

In [30]:
pct_Chg_feauture = next(filter(lambda feature_set: feature_set.name == 'pctChg_vars', cluster_group.group_params.X_feature_sets))
print(pct_Chg_feauture.cols)
cluster_group.train_all_rnns(pct_Chg_feauture.cols)

['pctChgopen', 'pctChghigh', 'pctChglow', 'pctChgclose', 'pctChgvolume', 'pctChgsma5', 'pctChgema5', 'pctChgsma10', 'pctChgema10', 'pctChgsma20', 'pctChgema20', 'pctChgsma30', 'pctChgema30', 'pctChgsma50', 'pctChgema50', 'pctChgsma100', 'pctChgema100', 'opHi', 'opLo', 'hiCl', 'loCl', 'hiLo', 'opCl', 'pctChgClOp', 'pctChgClLo', 'pctChgClHi']


2023-11-02 15:40:24.593495: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-11-02 15:40:24.593517: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: acet116-lnx-13
2023-11-02 15:40:24.593522: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: acet116-lnx-13
2023-11-02 15:40:24.593577: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.98.0
2023-11-02 15:40:24.593590: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.98.0
2023-11-02 15:40:24.593594: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.98.0
2023-11-02 15:40:25.182754: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor 

Epoch 1/100


2023-11-02 15:40:26.203285: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-02 15:40:26.204350: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-02 15:40:26.205102: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/2 [==============>...............] - ETA: 6s - loss: 4.3343

...

2023-11-02 15:48:43.562227: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-02 15:48:43.563195: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-02 15:48:43.563979: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

13/13 [==============================] - ETA: 0s - loss: 2.0065

2023-11-02 15:48:50.702894: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-11-02 15:48:50.704008: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-11-02 15:48:50.704837: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

13/13 [==============================] - 8s 134ms/step - loss: 2.0065 - val_loss: 0.1073
Epoch 2/100
13/13 [==============================] - 1s 40ms/step - loss: 1.2267 - val_loss: 0.1045
Epoch 3/100
13/13 [==============================] - 1s 40ms/step - loss: 0.8760 - val_loss: 0.1052
Epoch 4/100
13/13 [==============================] - 1s 40ms/step - loss: 0.7907 - val_loss: 0.1054
Epoch 5/100
13/13 [==============================] - 1s 39ms/step - loss: 0.6184 - val_loss: 0.1069
Epoch 6/100
13/13 [==============================] - 1s 40ms/step - loss: 0.5862 - val_loss: 0.1077
Epoch 7/100
13/13 [==============================] - 1s 39ms/step - loss: 0.4805 - val_loss: 0.1123
Epoch 8/100
13/13 [==============================] - 1s 40ms/step - loss: 0.4973 - val_loss: 0.1103
Epoch 9/100
13/13 [==============================] - 1s 39ms/step - loss: 0.4869 - val_loss: 0.1092
Epoch 10/100
13/13 [==============================] - 1s 39ms/step - loss: 0.3949 - val_loss: 0.1121
Epoch 11/1

KeyboardInterrupt: 

In [96]:
# # cluster_features = ['sumPctChgclose_1','sumPctChgclose_3','sumPctChgclose_6']
# cluster_features = ['sumPctChgclose_2_cumulative']
# print(stockSequence.group_params.X_feature_dict)
# cluster_indices = [stockSequence.group_params.X_feature_dict[feature] for feature in cluster_features]
# X_cluster = utils.create_Cluster_Seq(X_train,cluster_indices)

In [97]:
# import math
# n_clusters = math.ceil(math.sqrt(len(X_cluster))) // 3
# # A good rule of thumb is choosing k as the square root of the number of points in the training data set in kNN

# km = TimeSeriesKMeans(n_clusters=n_clusters)

# labels = km.fit_predict(X_cluster)

In [98]:
# reload(utils)
# threshold_factor = 1 # Adjust as necessary

# X_cluster_adj, X_train_adj, y_train_adj, labels_adj = utils.remove_outliers(X_cluster,X_train,y_train,labels,km,threshold_factor)
# display(X_cluster.shape)
# display(X_train.shape)
# display(y_train.shape)
# display(X_cluster_adj.shape)
# display(X_train_adj.shape)
# display(y_train_adj.shape)
# display(labels_adj.shape)

In [ ]:
# reload(utils)
# utils.visualizeData(labels_adj,X_cluster_adj)


In [ ]:
# from keras.layers import RepeatVector, TimeDistributed

# def create_model():
#     # Encoder
#     model_lstm = Sequential()
    
#     model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True, input_shape=(None, len(X_train_adj[0][0]))))
#     model_lstm.add(BatchNormalization())
#     model_lstm.add(Dropout(0.2))

#     model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True))
#     model_lstm.add(BatchNormalization())
#     model_lstm.add(Dropout(0.2))

#     model_lstm.add(LSTM(units=100, activation='tanh',return_sequences=True))
#     model_lstm.add(BatchNormalization())
#     model_lstm.add(Dropout(0.2))

#     model_lstm.add(LSTM(units=100, activation='tanh'))
#     model_lstm.add(BatchNormalization())
#     model_lstm.add(Dropout(0.2))
    
#     # Repeat the encoder output (which is the last hidden state) 
#     # for 'n' times where 'n' is the number of prediction steps
#     model_lstm.add(RepeatVector(6))  # Assuming you are predicting for 6 steps

#     # Decoder
#     model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True))
#     model_lstm.add(BatchNormalization())
#     model_lstm.add(Dropout(0.2))

#     model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True))
#     model_lstm.add(BatchNormalization())
#     model_lstm.add(Dropout(0.2))
    
#     model_lstm.add(TimeDistributed(Dense(1)))  # Predict one value for each time step

#     optimizer = Adam(learning_rate=0.001)
#     lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_lr=1e-5)

#     model_lstm.compile(optimizer=optimizer, loss="mse")
#     return model_lstm

In [ ]:
# # display(X_train[0:1,:,-1])
# X_cluster_test = utils.create_Cluster_Seq(X_test,cluster_indices)
# # display(X_cluster_test.shape)

# cluster_assignments = km.predict(X_cluster_test)


# for lab in set(labels_adj): 
#     filtered_x_train, filtered_y_train = utils.filter_clusters(labels_adj, lab, X_train_adj, y_train_adj)


#     # values, counts = np.unique(cluster_assignments, return_counts=True)

#     # # Display the results
#     # for value, count in zip(values, counts):
#     #     print(f"Value: {value}, Count: {count}")

#     X_cluster_test_adj,X_test_adj, y_test_adj, labels_test_adj = utils.remove_outliers(X_cluster_test,X_test,y_test,cluster_assignments,km,threshold_factor)


#     filtered_x_test, filtered_y_test = utils.filter_clusters(labels_test_adj, lab, X_test_adj, y_test_adj)
    
#     # display(X_cluster_test_adj.shape)
#     # display(filtered_x_test.shape)
#     # display(filtered_y_test.shape)

#     if len(filtered_x_test) == 0:
#         continue

#     # display(filtered_x_train.shape)
#     model_lstm = create_model()
#     print(filtered_x_test.shape)
#     print(filtered_y_test.shape)

#     y_train_reshaped = np.expand_dims(y_train, axis=-1)
#     y_test_reshaped = np.expand_dims(y_test, axis=-1)
    
#     model_lstm.fit(filtered_x_train, filtered_y_train, batch_size=30, epochs=300, verbose=1)

    
    
#     predicted_stock_price = model_lstm.predict(filtered_x_test)
#     predicted_stock_price = np.squeeze(predicted_stock_price, axis=-1)
#     print(predicted_stock_price.shape)



#     # predicted_stock_price = predictiveScaler.inverse_transform(predicted_stock_price)
#     # filtered_y_test = predictiveScaler.inverse_transform(filtered_y_test)

#     # Assuming predicted_stock_price has two columns
#     results = pd.DataFrame({
#         'predicted_1': predicted_stock_price[:, 0],
#         'predicted_2': predicted_stock_price[:, 1],
#         'predicted_3': predicted_stock_price[:, 2],
#         'predicted_4': predicted_stock_price[:, 3],
#         'predicted_5': predicted_stock_price[:, 4],
#         'predicted_6': predicted_stock_price[:, 5],
#         'real_1': filtered_y_test[:, 0],
#         'real_2': filtered_y_test[:, 1],
#         'real_3': filtered_y_test[:, 2],
#         'real_4': filtered_y_test[:, 3],
#         'real_5': filtered_y_test[:, 4],
#         'real_6': filtered_y_test[:, 5]
#     })

#     # Assuming a correct prediction means both predicted features have the same sign as their corresponding real values
#     results['same_1'] = ((results['predicted_1'] > 0) & (results['real_1'] > 0)) | ((results['predicted_1'] < 0) & (results['real_1'] < 0))
#     results['same_2'] = ((results['predicted_2'] > 0) & (results['real_2'] > 0)) | ((results['predicted_2'] < 0) & (results['real_2'] < 0))
#     results['same_3'] = ((results['predicted_3'] > 0) & (results['real_3'] > 0)) | ((results['predicted_3'] < 0) & (results['real_3'] < 0))
#     results['same_4'] = ((results['predicted_4'] > 0) & (results['real_4'] > 0)) | ((results['predicted_4'] < 0) & (results['real_4'] < 0))
#     results['same_5'] = ((results['predicted_5'] > 0) & (results['real_5'] > 0)) | ((results['predicted_5'] < 0) & (results['real_5'] < 0))
#     results['same_6'] = ((results['predicted_6'] > 0) & (results['real_6'] > 0)) | ((results['predicted_6'] < 0) & (results['real_6'] < 0))
#     accuracy1 = results['same_1'].sum() / len(results) * 100
#     accuracy2 = results['same_2'].sum() / len(results) * 100
#     accuracy3 = results['same_3'].sum() / len(results) * 100
#     accuracy4 = results['same_4'].sum() / len(results) * 100
#     accuracy5 = results['same_5'].sum() / len(results) * 100
#     accuracy6 = results['same_6'].sum() / len(results) * 100

#     # Construct the string
#     output_string = (
#         "Cluster Number: " + str(lab) +
#         " \nAccuracy1D " + str(accuracy1) + " PredictedRet: " + str(results['predicted_1'].mean()) + " ActRet " + str(results['real_1'].mean() ) +
#         " \nAccuracy2D " + str(accuracy2) + " PredictedRet: " + str(results['predicted_2'].mean()) + " ActRet " + str(results['real_2'].mean() ) +
#         " \nAccuracy3D " + str(accuracy3) + " PredictedRet: " + str(results['predicted_3'].mean()) + " ActRet " + str(results['real_3'].mean() ) +
#         " \nAccuracy4D " + str(accuracy4) + " PredictedRet: " + str(results['predicted_4'].mean()) + " ActRet " + str(results['real_4'].mean() ) +
#         " \nAccuracy5D " + str(accuracy5) + " PredictedRet: " + str(results['predicted_5'].mean()) + " ActRet " + str(results['real_5'].mean() ) +
#         " \nAccuracy6D " + str(accuracy6) + " PredictedRet: " + str(results['predicted_6'].mean()) + " ActRet " + str(results['real_6'].mean() ) +
#         " Test set length: " + str(len(filtered_y_test)) + "\n"
#     )

#     # Write the string to a file
#     with open('fuck.txt', 'a') as f:
#         f.write(output_string)






/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/numpy/core/_methods.py:265: RuntimeWarning:

Degrees of freedom <= 0 for slice

/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/numpy/core/_methods.py:223: RuntimeWarning:

invalid value encountered in divide

/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/numpy/core/_methods.py:257: RuntimeWarning:

invalid value encountered in double_scalars

2023-10-18 20:29:30.757084: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-18 20:29:30.757117: I tensorflow/compiler/xla/str

(69, 10, 81)
(69, 6)
Epoch 1/300


2023-10-18 20:29:32.127643: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-18 20:29:32.128624: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-18 20:29:32.129320: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

13/13 [==============================] - 6s 34ms/step - loss: 2.0318
Epoch 2/300
13/13 [==============================] - 0s 33ms/step - loss: 1.4424
Epoch 3/300
13/13 [==============================] - 0s 33ms/step - loss: 0.9040
Epoch 4/300
13/13 [==============================] - 0s 33ms/step - loss: 0.8543
Epoch 5/300
13/13 [==============================] - 0s 33ms/step - loss: 0.7150
Epoch 6/300
13/13 [==============================] - 0s 33ms/step - loss: 0.6277
Epoch 7/300
13/13 [==============================] - 0s 33ms/step - loss: 0.5982
Epoch 8/300
13/13 [==============================] - 0s 33ms/step - loss: 0.5874
Epoch 9/300
13/13 [==============================] - 0s 33ms/step - loss: 0.5316
Epoch 10/300
13/13 [==============================] - 0s 33ms/step - loss: 0.4584
Epoch 11/300
13/13 [==============================] - 0s 33ms/step - loss: 0.4312
Epoch 12/300
13/13 [==============================] - 0s 32ms/step - loss: 0.4289
Epoch 13/300
13/13 [=================

2023-10-18 20:31:41.208787: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-18 20:31:41.209967: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-18 20:31:41.210615: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

(18, 10, 81)
(18, 6)
Epoch 1/300


2023-10-18 21:18:54.970790: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-18 21:18:54.971761: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-18 21:18:54.972520: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

5/5 [==============================] - 6s 31ms/step - loss: 2.7985
Epoch 2/300
5/5 [==============================] - 0s 31ms/step - loss: 2.4691
Epoch 3/300
5/5 [==============================] - 0s 31ms/step - loss: 1.6993
Epoch 4/300
5/5 [==============================] - 0s 31ms/step - loss: 1.4584
Epoch 5/300
5/5 [==============================] - 0s 31ms/step - loss: 1.4673
Epoch 6/300
5/5 [==============================] - 0s 31ms/step - loss: 1.1239
Epoch 7/300
5/5 [==============================] - 0s 31ms/step - loss: 1.2261
Epoch 8/300
5/5 [==============================] - 0s 31ms/step - loss: 0.9473
Epoch 9/300
5/5 [==============================] - 0s 31ms/step - loss: 1.0794
Epoch 10/300
5/5 [==============================] - 0s 31ms/step - loss: 0.9555
Epoch 11/300
5/5 [==============================] - 0s 31ms/step - loss: 0.8402
Epoch 12/300
5/5 [==============================] - 0s 31ms/step - loss: 0.7327
Epoch 13/300
5/5 [==============================] - 0s 31ms/s

2023-10-18 21:19:45.860929: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-18 21:19:45.861853: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-18 21:19:45.862549: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 1s 1s/step
(18, 6)
